In [10]:
# Import necessary modules
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import seaborn as sns

# Step 1: Generate Data

In [11]:
# Define constants for data generation
N_m = 5000  # Number of measurements
k = 0.5  # Decay constant
L = np.pi  # Spatial domain bound
T_end = 1  # Time domain bound
noise_level = 0.01  # Noise level for observations
N_sb = 128  # Number of boundary points in space
N_tb = 128  # Number of boundary points in time
N_int = int(100**2)  # Number of interior points

# Function to calculate the exact solution
def u_star(x, t):
    return np.sin(x) * np.exp(-k * t)

In [12]:
# Initialize random number generator
rng = np.random.default_rng(seed=1)

# Generate measurement points in space and time
x_vals = rng.uniform(0, L, N_m)
t_vals = rng.uniform(0, T_end, N_m)

# Generate sensor points in space and time
x_vals_sensor = rng.uniform(0, L, N_m)
t_vals_sensor = rng.uniform(0, T_end, N_m)
u_true = u_star(x_vals_sensor, t_vals_sensor) 

# Add noise to the true observations
mea_sig = noise_level * np.std(u_true)
y_obs_int = u_true + rng.normal(0, mea_sig, u_true.shape)

# Generate boundary condition points when t = 0 and t = 1
t_vals_tb = np.random.choice([0, T_end], N_tb, replace=True)
x_vals_tb = rng.uniform(0, L, N_tb)
u_vals_tb = u_star(x_vals_tb, t_vals_tb)

# Generate boundary condition points when x = 0 or L
t_vals_sb = rng.uniform(0, T_end, N_sb)
x_vals_sb = np.random.choice([0, L], N_sb, replace=True)
u_vals_sb = u_star(x_vals_sb, t_vals_sb)

# Generate interior points
x_vals_int = rng.uniform(0, L, N_int)
t_vals_int = rng.uniform(0, T_end, N_int)

# Generating the required data arrays
d_tb = np.column_stack((x_vals_tb, t_vals_tb, u_vals_tb))
d_sb = np.column_stack((x_vals_sb, t_vals_sb, u_vals_sb))
data = np.column_stack((x_vals_sensor, t_vals_sensor, y_obs_int))
inter = np.column_stack((x_vals_int, t_vals_int))


## Import functions and classes from our scripts

In [13]:
from model import get_pred_nn
from sparse_net_util import (
    flatten, unflatten_trainable_variables, tf_flatten, sparse_weight
)
from train_network import train

# Step 2: Define Hyperparameters

In [18]:

N_M = data.shape[0]
N_int = inter.shape[0]
# uu, JJ_ratio, rho_0 and rho_1 are parameters used for tuning sparsity of the network
# rho is prior for theta
# alpha is control importance of pde term
# sig_M2, sig_R2 and sig_B2 is to control relative weight on meansurement data, interior data and boundary data
hps = {
    'uu': 1.1,
    'JJ_ratio': 0.02,  
    'alpha': 1.0,
    'rho_0': N_M,
    'rho_1': 1,
    'rho': 1,
    'init_lr': 1e-10,
    'delta_prob': 1.0,
    'sig_M2': mea_sig**2,
    'sig_R2': 1,
    'sig_B2': 1
}

# Step 3: Train the Model

In [19]:
iterations = 400000  # You can adjust the number of iteration
print("Starting training process...")
Ws_list, theta_hist_keep, delta_list = train(iterations, hps, data, d_sb, d_tb, inter)
print("Training completed.")

  0%|          | 3/400000 [00:00<4:56:28, 22.49it/s]

Starting training process...


 10%|█         | 40002/400000 [29:28<4:25:02, 22.64it/s]

Iteration 40000: Loss = -27576.267578125, Theta = 0.5189424196715717


 20%|██        | 80004/400000 [58:57<3:55:32, 22.64it/s]

...

Iteration 320000: Loss = -3699.037841796875, Theta = 0.48319598680275305


 90%|█████████ | 360003/400000 [4:24:50<29:16, 22.78it/s]

Iteration 360000: Loss = -5013.40576171875, Theta = 0.505364133288279


100%|██████████| 400000/400000 [4:54:09<00:00, 22.66it/s]


Iteration 400000: Loss = -8445.9482421875, Theta = 0.471992451060153
Training completed.


In [36]:
theta_hist_keep = np.array(theta_hist_keep)

# Step 4: Analyze Result

In [20]:
np.mean(theta_hist_keep) 

0.48882250327521654

In [21]:
np.std(theta_hist_keep) 

0.025098192558921095

## calculate mean and std for target distribution

In [37]:
k = 0.5
def u_star(x, t):
    return np.sin(x)*np.exp(-k*t)
def du_dx_star(x,t):
    return np.cos(x)*np.exp(-k*t)
def d2u_dxx_star(x,t):
    return -np.sin(x)*np.exp(-k*t)
def du_dt_star(x,t):
    return -k*np.sin(x)*np.exp(-k*t)

from scipy.integrate import dblquad
# Set the integration limits
x_lower = 0
x_upper = np.pi
t_lower = 0
t_upper = 1



def fun_to_int(x,t):
    return (d2u_dxx_star(x,t))**2 

# Perform the double integral
integral_result, _ = dblquad(fun_to_int, t_lower, t_upper, lambda t: x_lower, lambda t: x_upper)
temp = (integral_result + (hps['sig_R2'])/(N_M**hps['alpha']))
inv_temp = (1/(temp))
Sig_star = ((hps['sig_R2'])/(N_M**hps['alpha'])) * inv_temp


In [38]:
theta_target_star = rng.normal(k, np.sqrt(Sig_star), len(theta_hist_keep))

In [39]:
theta_target_star

array([0.49846329, 0.49003444, 0.51609326, ..., 0.4967145 , 0.4931076 ,
       0.49011786])

## RMSE and $W_2$

In [41]:
import ot
ot.wasserstein_1d(theta_hist_keep, theta_target_star, p=2) 

0.0002479730305295602

In [43]:
np.mean((theta_hist_keep-k)**2) 

0.0007548557027572286

## Upper and lower bound for TVD

In [45]:
def tvd_upper_bound(mean0, mean1, std0, std1):
    """
    Calculate the upper bound for the total variation distance (TVD) using Pinsker's inequality
    between two normal distributions.
    
    Parameters:
    mean0 (float): Mean of the first normal distribution.
    std0 (float): Standard deviation of the first normal distribution.
    mean1 (float): Mean of the second normal distribution.
    std1 (float): Standard deviation of the second normal distribution.
    
    Returns:
    float: The upper bound for the TVD.
    """
    # Calculate KL divergence N0 || N1
    kl_01 = np.log(std1 / std0) + (std0**2 + (mean0 - mean1)**2) / (2 * std1**2) - 0.5
    
    # Calculate KL divergence N1 || N0
    kl_10 = np.log(std0 / std1) + (std1**2 + (mean1 - mean0)**2) / (2 * std0**2) - 0.5
    
    # Use the minimum KL divergence for Pinsker's inequality
    min_kl = min(kl_01, kl_10)
    
    # Calculate the upper bound for TVD using Pinsker's inequality
    out = np.sqrt(0.5 * min_kl)
    
    return min(1, out)

def tvd_lower_bound(mean0, mean1, std0, std1):
    """
    Calculate the lower bound for the total variation distance (TVD) using the Hellinger distance
    between two normal distributions.
    
    Parameters:
    mean0 (float): Mean of the first normal distribution.
    std0 (float): Standard deviation of the first normal distribution.
    mean1 (float): Mean of the second normal distribution.
    std1 (float): Standard deviation of the second normal distribution.
    
    Returns:
    float: The lower bound for the TVD.
    """
    # Calculate the Hellinger distance squared
    term1 = np.sqrt(2 * std0 * std1 / (std0**2 + std1**2))
    term2 = np.exp(-(mean0 - mean1)**2 / (4 * (std0**2 + std1**2)))
    h_squared = 1 - term1 * term2

    return h_squared

def tvd_upper_Hell(mean0, mean1, std0, std1):
    """
    Calculate the lower and upper bounds for the total variation distance (TVD) using the Hellinger distance
    between two normal distributions.
    
    Parameters:
    mean0 (float): Mean of the first normal distribution.
    std0 (float): Standard deviation of the first normal distribution.
    mean1 (float): Mean of the second normal distribution.
    std1 (float): Standard deviation of the second normal distribution.
    
    Returns:
    tuple: A tuple containing the lower bound and the upper bound for the TVD.
    """
    # Calculate the Hellinger distance squared
    term1 = np.sqrt(2 * std0 * std1 / (std0**2 + std1**2))
    term2 = np.exp(-(mean0 - mean1)**2 / (4 * (std0**2 + std1**2)))
    h_squared = 1 - term1 * term2

    # Hellinger distance
    h_distance = np.sqrt(h_squared)
    out = np.sqrt(2) * h_distance
    
    return out


In [46]:
mu_theta = np.mean(theta_hist_keep) 
std_theta = np.std(theta_hist_keep) 

In [47]:
tvd_upper_bound(mu_theta, k, std_theta, np.sqrt(Sig_star))

0.4057144773609908

In [48]:
tvd_lower_bound(mu_theta, k, std_theta, np.sqrt(Sig_star))

0.1084516542013445

In [49]:
# this is upper bound with Hellinger which usually larger than the one from KL
tvd_upper_Hell(mu_theta, k, std_theta, np.sqrt(Sig_star)) 

0.4657287927567814